In [41]:
import copy
from datetime import datetime

# Получаем индекс элемента в массиве
# Если элемент не найден, добавляем его в конец массива и возвращаем его номер
def getIndex(arr, val):
  try:
    return arr.index(val)
  except:
    arr.append(val)
    return len(arr) - 1

# Из переданного массива arr получаем запись, у которой поле key соответствует значению value
# Если такая запись не обнаружена, то создаем новую запись из объекта default, устанавливаем полу key в значение value и помещаем в массив
def getValueOrDefault(arr, key, value, default):
  res = next((x for x in arr if x[key] == value), None)
  if (res == None):
    res = copy.deepcopy(default)
    res[key] = value
    arr.append(res)
  return res

# Из строки даты (формат: дд.мм.гггг) получаем месяц
def getBdate(bdate):
  return datetime.strptime(bdate, '%d.%m.%Y').month

In [66]:
import numpy as np
import csv
import sys

input = []

with open('input.csv', newline='') as csvfile:
  inputreader = csv.reader(csvfile, delimiter=',', quotechar='"')
  for row in inputreader:
    input.append(row)

# Прекращаем работу приложения, если не было прочитано ни одной строки
# или первая строка (заголовок) не содержит ни одного столбца 
if (len(input) < 2):
  print('Датасет пустой. Приложение будет остановлено')
  sys.exit()

if (len(input[0]) == 0):
  print('Датасет пустой. Приложение будет остановлено')
  sys.exit()

# Выводим размер датасета. 
# len - 1, потому что первый столбец - это заголовок.
print('Датасет содержит %d записей' % (len(input) - 1))

# Объявляем переменные, в которые запишем номера столбцов. 
# Если в датасети номера столбцов поменяются местами или пропадут, мы автоматические это определим
# значение по умолчанию -1, т.к. нумерация массива с 0. -1 означает, что столбца нет в датасете
npp = fio = sex = bdate = ege = discip = school = city = prof = budget = -1

# Заполняем номера столбцов, сравнивая их с названием ячейки из первой строки датасета
for j in range(len(input[0])):
  col = input[0][j]
  if (col == '№п/п'): npp = j
  if (col == 'ФИО'): fio = j
  if (col == 'Пол'): sex = j
  if (col == 'Дата рождения'): bdate = j
  if (col == 'Баллы ЕГЭ'): ege = j
  if (col == 'Предметы ЕГЭ'): discip = j
  if (col == 'Учебное заведение'): school = j
  if (col == 'Населенный пункт по прописке'): city = j
  if (col == 'Специальность/направление'): prof = j
  if (col == 'Основание поступления'): budget = j

if (fio == -1):
  print('Столбец ФИО не определен в датасете. Приложение будет остановлено')
  sys.exit()

discipList = []
schoolList = []
cityList = []
profList = []
budgetList = []
peopleList = []

# Объект человека по умолчанию
DEFAULT_PEOPLE = {'fio':None,'sex':None,'bdate':None,'city':None,'school':None,
                  'egeMathem':0,'egeRussky':0,'egeInform':0,'egePhysic':0,'egeSociety':0,
                  'prof':None,'budget':None}

# Проходим по датасету и заполняем объект человека информацией. 
# т.к. в датасете каждая строя - это бал ЕГЭ, то на одного человека несколько строк в датасете
# поэтому их надо объединить в один объект
# при этом заменяем строковые поля характерным номером
for j in range(1, len(input)):
  p = getValueOrDefault(peopleList, 'fio', input[j][fio], DEFAULT_PEOPLE)
  
  if (sex != -1):
    if (input[j][sex] == 'Мужской'):
      p['sex'] = 0
    elif (input[j][sex] == 'Женский'):
      p['sex'] = 1
    else:
      print('ВНИМАНИЕ: Неизвестный пол %s' % input[j][sex])

  if (bdate != -1):
    p['bdate'] = getBdate(input[j][bdate])

  if (city != -1):
    p['city'] = getIndex(cityList, input[j][city])

  if (school != -1):
    p['school'] = getIndex(schoolList, input[j][school])

  if (prof != -1):
    p['prof'] = getIndex(profList, input[j][prof])

  if (budget != -1):
    p['budget'] = getIndex(budgetList, input[j][budget])

  if (ege != -1 and discip != -1):
    if (input[j][discip] == 'Русский язык'):
      p['egeRussky'] = int(input[j][ege])
    elif (input[j][discip] == 'Математика'):
      p['egeMathem'] = int(input[j][ege])
    elif (input[j][discip] == 'Физика'):
      p['egePhysic'] = int(input[j][ege])
    elif (input[j][discip] == 'Информатика и ИКТ'):
      p['egeInform'] = int(input[j][ege])
    elif (input[j][discip] == 'Обществознание'):
      p['egeSociety'] = int(input[j][ege])
    else:
      print('ВНИМАНИЕ: Неизвестный предмет ЕГЭ %s' % input[j][discip])

print('Всего человек в датасете: %d' % len(peopleList))



# Подготавливаем датасет для нейронной сети
# Для этого все значения необходимо привести к диапазону 0..1
# Баллы ЕГЭ делим на 100, т.к. 100 максимальный бал,
# город и школу делим на количество записей
# профессии мы не делим, т.к. это число у нас определяет номер активируемого нейрона
# дату рождения (номер месяца) делим на 12 (кол-во месяцев в году)
# пол при заполнении уже лежит в диапазоне 0..1 (0 - мужской, 1 - женский)

nnDataset = np.array([])
nnLabels = np.array([])

# Количество нейронов на входном слое
# При заполнении массива nnDataset необходимо, чтобы количество параметров совпадало с этим числом
INPUT_SIZE = 9

for j in range(len(peopleList)):
  p = peopleList[j]
  nnDataset = np.append(nnDataset, [
                    p['egeRussky'] / 100, 
                    p['egeMathem'] / 100, 
                    p['egePhysic'] / 100,
                    p['egeInform'] / 100,
                    p['egeSociety'] / 100,
                    p['sex'],
                    p['bdate'] / 12,
                    p['city'] / len(cityList),
                    p['school'] / len(schoolList)
                    ])
  
  nnLabels = np.append(nnLabels, p['prof'])

# приводим датасет к двумерному виду. 
nnDataset = nnDataset.reshape(-1, INPUT_SIZE)

train = copy.deepcopy(nnDataset)
train_labels = copy.deepcopy(nnLabels)

Датасет содержит 1310 записей
Всего человек в датасете: 443


In [64]:
# TensorFlow и tf.keras
import tensorflow as tf
from tensorflow import keras

model_first = keras.Sequential([
  #  слои модели    
  # Flatten преобразует в одномерный массив
  # keras.layers.Flatten(input_shape=(maxShape, 13)),

  keras.layers.InputLayer(INPUT_SIZE),

  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(64, activation='relu'),

  # Последний слой определяет профессию человека
  # Количество нейронов должно соответствовать количеству профессий
  keras.layers.Dense(len(profList), activation='softmax')
])

model_first.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_first.fit(train, train_labels, epochs=100, batch_size=32)


check = 123
checkDigit = (np.expand_dims(nnDataset[check],0))
predictions_single = model_first.predict(checkDigit)
predictProf = np.argmax(predictions_single[0])

print('Predict %d. Real: %d' % (predictProf, nnLabels[check]))

Epoch 1/100
14/14 [==============================] - 0s 2ms/step - loss: 2.9374 - accuracy: 0.1625
Epoch 2/100
14/14 [==============================] - 0s 2ms/step - loss: 2.7564 - accuracy: 0.1964
Epoch 3/100
14/14 [==============================] - 0s 2ms/step - loss: 2.5653 - accuracy: 0.1919
Epoch 4/100
14/14 [==============================] - 0s 1ms/step - loss: 2.3994 - accuracy: 0.2054
Epoch 5/100
14/14 [==============================] - 0s 1ms/step - loss: 2.2612 - accuracy: 0.3093
Epoch 6/100
14/14 [==============================] - 0s 1ms/step - loss: 2.1295 - accuracy: 0.3341
Epoch 7/100
14/14 [==============================] - 0s 2ms/step - loss: 2.0134 - accuracy: 0.3318
Epoch 8/100
14/14 [==============================] - 0s 2ms/step - loss: 1.9213 - accuracy: 0.3431
Epoch 9/100
14/14 [==============================] - 0s 2ms/step - loss: 1.8536 - accuracy: 0.3544
Epoch 10/100
14/14 [==============================] - 0s 2ms/step - loss: 1.8106 - accuracy: 0.3612
Epoch 11/